<a href="https://colab.research.google.com/github/iceman67/-Python/blob/master/seoul_bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 서울 자전거 대여 


In [ ]:
import pandas as pd

In [ ]:
bike_seoul = "bike_seoul_2020.csv"
df = pd.read_csv(bike_seoul,parse_dates=['tm'] ,thousands = ',') 



* ,를 없애고 숫자로 변환 
df['count'] = df['count'].str.replace(',', '').astype(int)

* 여러 dataFrame 을 통합함
frames = [bike_df0, bike_df1, bike_df2, bike_df3]
df = pd.concat(frames)

In [ ]:
type(df)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df['weekday'] = df['tm'].dt.dayofweek


In [ ]:

# 요일이름 얻기  
df['weekday_str'] = df['tm'].dt.day_name()
print(df.columns)

In [ ]:
df.head()

In [ ]:
df['month'] = df['tm'].dt.month

In [ ]:
df.columns

In [ ]:
df.rename(columns = {'count ':'count'},inplace=True)

In [ ]:
sub_df = df[ ['tm', 'month', 'count'] ]

In [ ]:
sub_df.groupby(['month']).mean()

In [ ]:
df.head()

In [ ]:
df[['tm', 'count']][1:3]

In [ ]:
sub_df = df[ ['tm', 'weekday', 'count'] ]

grouped = sub_df.groupby('weekday').sum()['count']

In [ ]:
grouped

In [ ]:
grouped.plot(kind='bar')

In [ ]:
sub_df = df[ ['tm', 'weekday_str', 'count'] ]
grouped = sub_df.groupby('weekday_str').sum()['count']

In [ ]:
grouped.plot(kind='bar')

In [ ]:
grouped = df.groupby('month').sum()

In [ ]:
grouped

In [ ]:
grouped = df.groupby('month').sum()['count']

In [ ]:
grouped

In [ ]:
grouped.plot(kind='bar')

### ASOS 108

* 참고
ADS_2022_OpenAPI_ASOS_DA.ipynb (5주)

In [ ]:
def set_url(key, numOfRows, startDt, endDt, stnId, ctype):
    base = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    url =f"{base}?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType={ctype}&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"

    return url


In [ ]:
import requests

def get_contents(URL):
    # GET 메소드로 HTTP 요청을 전달함
    response = requests.get(URL)
    print ("status code : ", response.status_code)
    # 바이트를 문자열로 변환 
    result = response.content.decode('utf-8')

    return response

In [ ]:
def get_asos_info(response):
    # 문자열 JSON 형식을 딕셔너리 객체로 변환 
    #res = json.loads(result) # jon 라이브러리 사용
    res = response.json() # requests 라이브러리 사용
    items = res['response']['body']['items'] 
    item = items['item'] # [.... ] 배열 
    print (len(item))

    return item


In [ ]:
serviceKey = 'y80jnESQZu1%2B%2BKrpWpkGrnZ96%2FhiBicuIH%2F3SeO0u10CK9rglO3nqmwetj8%2BRHj%2F1NWUUis4aeGnUMk1CFUYRQ%3D%3D'
numOfRows = 400
startDt = 20200101
endDt = 20201231
stnId =108 # 지역
#URL=f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"
#URL

In [ ]:
URL = set_url(serviceKey, numOfRows, startDt, endDt, stnId, "JSON")
result = get_contents(URL)
item = get_asos_info(result)
# 첫번쨰 기상정보


In [ ]:
# 2020 서울 날씨
import csv

max_ws = 'ASOS_108.csv'
f = open(f'{max_ws}','w', encoding="cp949")
schema = ['stnId','tm', 'avgTa', 'avgPa', 'maxWS']

wr = csv.writer(f)

# 리스트 자료형을 활용하여 헤더를 출력함 
wr.writerow([g for g in schema])   

# 수집된 자료의 특징 자료만을 추출하여 출력을 위한 행을 만들어 출력함  
for i in range(len(item)):
     stnId = item[i]['stnId'] 
     tm = item[i]['tm'] 
     avgTa = item[i]['avgTa']
     avgPa = item[i]['avgPa']
     maxWs = item[i]['maxWs'] 
     wr.writerow ([ stnId, tm, avgTa, avgPa, maxWs])

f.close()

* 함수로 구성

In [ ]:
import csv

def write_cvs(fname,  schema,item):
    fname = f'{fname}.csv'
    f = open(fname,'w', encoding="cp949")
    wr = csv.writer(f)
    # 리스트 자료형을 활용하여 헤더를 출력함 
    wr.writerow([g for g in schema])  

    # 수집된 자료의 특징 자료만을 추출하여 출력을 위한 행을 만들어 출력함  
    for i in range(len(item)):
        record = []
        for j in schema:
            record.append(item[i][j])
        wr.writerow (record)
    f.close()

In [ ]:
asos_file_path= 'ASOS_108'
schema = ['stnId','tm', 'avgTa', 'avgPa', 'maxWs']

write_cvs(asos_file_path, schema, item)

In [ ]:
# ASOS_108.csv 
df_asos = pd.read_csv(f'{asos_file_path}.csv',parse_dates=['tm'])

In [ ]:
merge_df = pd.merge(df_asos,df, how='outer',on='tm')


In [ ]:
import numpy as np

merge_df['w_cond'] = np.where(merge_df['avgPa'] > 1000, True, False)

In [ ]:
wgroup = merge_df.groupby(merge_df['w_cond'], as_index=False).sum()

In [ ]:
wgroup['w_cond_str'] = wgroup['w_cond'].map({True: 'Good weather', False: 'Bad weather'}) 

In [ ]:
import matplotlib.pyplot as plt

plt.bar ( wgroup['w_cond'], wgroup['count'])

In [ ]:
import matplotlib.pyplot as plt

plt.bar ( wgroup['w_cond_str'], wgroup['count'])

* 좋은 날씨에 자전거 대여를 많이 하는 것을 확인할 수 있었음